In [28]:
from surprise import Reader, Dataset, SVD,accuracy,NormalPredictor
from surprise.model_selection import train_test_split,cross_validate
import pandas as pd

In [17]:
df = pd.read_csv('products.csv')
df['product'] = df['product'].drop_duplicates()
df = df.dropna()
df.head()

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [15]:
df_new = df.reset_index()
products = df_new['product']
indices = pd.Series(df_new.index, index=df_new['product'])
df_new[['description','product']]

,description,product
0,This Product contains Garlic Oil that is known...,Garlic Oil - Vegetarian Capsule 500 mg
1,"Each product is microwave safe (without lid), ...",Water Bottle - Orange
2,"A perfect gift for all occasions, be it your m...","Brass Angle Deep - Plain, No.2"
3,Multipurpose container with an attractive desi...,Cereal Flip Lid Container/Storage Jar - Assort...
4,Nivea Creme Soft Soap gives your skin the best...,Creme Soft Soap - For Hands & Body
...,...,...
16019,This round toilet brush is made up of virgin q...,Toilet Cleaning Brush - Round With Holder (Big)
16020,Organic Tattva Garam masala is a famous spice ...,Organic Powder - Garam Masala
16021,Layerr brings you Wottagirl Classic fragrant b...,"Wottagirl! Perfume Spray - Heaven, Classic"
16022,We have taken the richness of Sweet Potatoes (...,Peri-Peri Sweet Potato Chips


In [40]:
data = Dataset.load_from_df(df_new[['product', 'category','rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)

In [41]:
algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

In [42]:
accuracy.rmse(predictions)

RMSE: 0.7555


0.7555280505345193

In [43]:
cross_validate(NormalPredictor(), data, cv=2)

{'test_rmse': array([1.03107607, 1.01659378]),
 'test_mae': array([0.79442108, 0.78413511]),
 'fit_time': (0.01726388931274414, 0.013636589050292969),
 'test_time': (0.0990598201751709, 0.09212899208068848)}

In [30]:
def generate_recommendation(model, user_id, ratings_df, movies_df, n_items):
   # Get a list of all movie IDs from dataset
   movie_ids = ratings_df["movieId"].unique()
 
   # Get a list of all movie IDs that have been watched by user
   movie_ids_user = ratings_df.loc[ratings_df["userId"] == user_id, "movieId"]
    # Get a list off all movie IDS that that have not been watched by user
   movie_ids_to_pred = np.setdiff1d(movie_ids, movie_ids_user)
 
   # Apply a rating of 4 to all interactions (only to match the Surprise dataset format)
   test_set = [[user_id, movie_id, 4] for movie_id in movie_ids_to_pred]
 
   # Predict the ratings and generate recommendations
   predictions = model.test(test_set)
   pred_ratings = np.array([pred.est for pred in predictions])
   print("Top {0} item recommendations for user {1}:".format(n_items, user_id))
   # Rank top-n movies based on the predicted ratings
   index_max = (-pred_ratings).argsort()[:n_items]
   for i in index_max:
       movie_id = movie_ids_to_pred[i]
       print(movies_df[movies_df["movieId"]==movie_id]["title"].values[0], pred_ratings[i])

In [50]:
testset[0]

('Double Chicken Pepperoni Pizza', 'Snacks & Branded Foods', 3.5)

In [59]:
predictions = algo.predict(uid='Double Chicken Pepperoni Pizza',iid='Snacks & Branded Foods')

In [60]:
predictions

Prediction(uid='Double Chicken Pepperoni Pizza', iid='Snacks & Branded Foods', r_ui=None, est=3.9779736979133835, details={'was_impossible': False})